<a href="https://colab.research.google.com/github/newton143/Business_Forecasting_Project/blob/main/Final_Version_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import requests
import io
import re
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import math
import plotly.express as px

In [6]:
# File Pre-Processing and Data Cleanup
def preprocess_excel_file(url, year):
    try:
        response = requests.get(url)
        response.raise_for_status()
        excel_data = io.BytesIO(response.content)
        df = pd.read_excel(excel_data, sheet_name='Sign In Out Times Report')

        df = df.iloc[4:]
        new_header = df.iloc[0]
        df = df[1:]
        print("Stage0")
        display(df.head())
        print(df.shape)
        df.columns = new_header

        def rename_columns(df_format):
            count = 7
            old_col_name = None
            for col in df_format.columns[7:]:  # 'JAN 01' is the 8th column (index 7)
              if pd.isna(col):
                if old_col_name:
                  new_col_name = f"{old_col_name} - OUT"
                  df_format.columns.values[count] = new_col_name
              else:
                new_col_name = f"{col} - IN"
                old_col_name = col
                df_format.rename(columns={col: new_col_name}, inplace=True)

              count += 1
              if new_col_name == 'Dec 31 - OUT':
                break
            return df_format

        df = rename_columns(df)
        print("Stage1: Renaming Column from 7 to 737 as JAN 01 to Dec 31")
        display(df.head())
        print(df.shape)

        df = df.dropna(subset=['Record ID'])
        df = df[~((df['Student Status'] != 'Active') & (df['Record ID'].isna()))]
        print("Stage2: Filter only Student Status = Active")
        display(df.head())
        print(df.shape)

        columns_to_drop = ['First Name', 'Last Name', 'External Student ID', 'Student Status', 'Tags']
        df = df.drop(columns=columns_to_drop, errors='ignore')
        print("Stage3: Drop Unwanted features like ['First Name', 'Last Name', 'External Student ID', 'Student Status', 'Tags'] ")
        display(df.head())
        print(df.shape)

        def remove_rows_with_no_data(df):
            rows_to_drop = []
            for index, row in df.iterrows():
                if pd.isna(row.iloc[2:]).all():
                    rows_to_drop.append(index)
            return df.drop(rows_to_drop)

        df = remove_rows_with_no_data(df)
        print("Stage4: Remove Rows with no data")
        display(df.head())
        print(df.shape)

        def reshape_data(df):
            date_cols = df.columns[2:]
            reshaped_data = []
            for index, row in df.iterrows():
                record_id = row['Record ID']
                room = row['Room']
                for date_col in date_cols:
                    in_time = row[date_col]
                    col_index = date_cols.get_loc(date_col)
                    if 'IN' in date_col and col_index < len(date_cols) -1:
                        out_time_col = date_cols[col_index + 1]
                        out_time = row[out_time_col]
                        reshaped_data.append([record_id, room, date_col, in_time, out_time])
            return pd.DataFrame(reshaped_data, columns=['Record ID', 'Room', 'Date', 'IN', 'OUT'])

        df = reshape_data(df)
        print("Stage5: Reshape with needed features 'Record ID', 'Room', 'Date', 'IN', 'OUT' ")
        display(df.head())
        print(df.shape)

        def extract_time_new(time_str):
          if pd.isna(time_str):
              return ""
          match = re.search(r'(\d+:\d+\s*[APap][Mm])', str(time_str))
          if match:  # Check if a match was found
            return match.group(1)  # Access group(1) only if match is not None
          else:
            return ""

        def extract_time(time_str):
            if pd.isna(time_str):
                return ""
            match = re.search(r'(\d+:\d+\s*[APap][Mm])', str(time_str))
            return match.group(1) if match else ""

        df['IN'] = df['IN'].apply(extract_time)
        df['OUT'] = df['OUT'].apply(extract_time)

        df = df.dropna(subset=['IN', 'OUT'], how='all')
        df = df[~((df['IN'] == "") & (df['OUT'] == ""))]

        df['Date'] = df['Date'].str.replace(' - IN', f" {year}")
        print("Stage6: Parse Timestamp from IN and OUT")
        display(df.head())
        df.shape

        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the file: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [7]:
# running the dataset URLs through our process_excel_file function and merging them into one dataset
urls = [
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/ECEC%202022%20Student%20Sign%20In%20and%20Out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/ECEC%202023%20Student%20Sign%20In%20and%20Out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/ECEC%202024%20Student%20Sign%20In%20and%20Out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/ECEC%202025%2001012025-02282025%20Student%20Sign%20In%20and%20Out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/Spellman%20CDC%202022%20Student%20Sign%20in%20and%20out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/Spellman%20CDC%202023%20Student%20Sign%20in%20and%20out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/Spellman%20CDC%202024%20Student%20Sign%20in%20and%20out.xlsx?raw=true',
    'https://github.com/newton143/Business_Forecasting_Project/blob/main/Spellman%20CDC%202025%2001012025-02282025%20Student%20Sign%20in%20and%20out.xlsx?raw=true'
    ]
years = [
    '2022',
    '2023',
    '2024',
    '2025',
    '2022',
    '2023',
    '2024',
    '2025'
    ]

all_dfs = all_dfs = []
for i, url in enumerate(urls):
    time.sleep(20)
    result_df = preprocess_excel_file(url, years[i])
    if result_df is not None:
        all_dfs.append(result_df)

final_df = pd.concat(all_dfs, ignore_index=True)

# adding a location column to differentiate between ECEC and Spellman buildings
def add_location_column(df):
    ecec_rooms = ['Pennie Preschool 1', 'Henry Multi-Age', 'Grampy Tom Multi-Age', 'Henry Infants', 'Pennie Infants', 'Pre-K1', 'Pre-K2', 'Pennie Toddlers', 'Henry Toddlers', 'Grampy Tom Toddlers', 'Grampy Tom Preschool', 'Camp']
    df['Location'] = df['Room'].apply(lambda x: 'ECEC' if x in ecec_rooms else 'Spellman')
    return df

final_df = add_location_column(final_df)

# adding an age group column that corresponds to the age-group mapping in the Data Dictionary file
room_mapping = {
    'Good Night Moon': 'Infant',
    'Goodnight Moon': 'Infant',
    'Panda Bear': 'Toddler',
    'House of Pooh': 'Infant',
    'Llama Llama': 'Multi-Age',
    'Caterpillars': 'Multi-Age',
    'Rabbits': 'Toddler',
    'Wild Things': 'Preschool',
    'Monkeys': 'Toddler',
    'Rainbow Fish': 'Preschool',
    'Dinosaurs': 'Pre-K',
    'Pre-K1': 'Pre-K',
    'Pre-K2': 'Pre-K',
    'Henry Toddlers': 'Toddler',
    'Pennie Preschool 1': 'Preschool',
    'Grampy Tom Preschool': 'Preschool',
    'Grampy Tom Toddlers': 'Toddler',
    'Henry Multi-Age': 'Multi-Age',
    'Grampy Tom Multi-Age': 'Multi-Age',
    'Pennie Toddlers': 'Toddler',
    'Henry Infants': 'Infant',
    'Camp': 'Multi-Age',
    'Pennie Infants': 'Infant',
    'Dinosaur Stomp': 'Pre-K',
    'Pandas': 'Toddler',
    'Llamas Llamas': 'Multi-Age',
    'House Pooh': 'Infant',
    'Hungry Caterpillars': 'Multi-Age'
}

final_df['AgeGroup'] = final_df['Room'].map(room_mapping)
final_df = final_df[['Record ID', 'Location', 'Room', 'AgeGroup', 'Date', 'IN', 'OUT']]

# changing 'IN' and 'OUT' columns to pandas datetime
final_df['IN'] = pd.to_datetime(final_df['IN'], format='%I:%M %p', errors='coerce').dt.time
final_df['OUT'] = pd.to_datetime(final_df['OUT'], format='%I:%M %p', errors='coerce').dt.time

# filtering out rows with NaN in either IN or OUT columns
filtered_df = final_df.dropna(subset=['IN', 'OUT'])

# creating time 30 min time interval columns within building hours of opperation
time_intervals = pd.date_range(start='06:00:00', end='19:00:00', freq='30min').time

# filling out the time interval columns with binary values to indicate attendance
def create_binary_columns(df):
    for interval in time_intervals:
        start_time = interval
        end_time = (pd.to_datetime(interval, format='%H:%M:%S') + pd.Timedelta(minutes=30)).time()
        col_name = f'Presence_{interval.strftime("%H%M")}'
        df[col_name] = 0

        df.loc[(df['IN'] <= start_time) & (df['OUT'] > start_time), col_name] = 1

    return df

final_df = create_binary_columns(filtered_df.copy())

#adding Day of Week for another level to group the forecast by
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df.insert(final_df.columns.get_loc('Date') + 1, 'Day of the Week', final_df['Date'].dt.day_name())

final_df


Stage0


,"Multiple Rooms Sign In Out Times Report for 01 January, 2022 - 31 December, 2022",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 727,Unnamed: 728,Unnamed: 729,Unnamed: 730,Unnamed: 731,Unnamed: 732,Unnamed: 733,Unnamed: 734,Unnamed: 735,Unnamed: 736
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IN,OUT,IN,...,IN,OUT,IN,OUT,IN,OUT,IN,OUT,IN,OUT
6,NaN,NaN,KL100349,NaN,Active,Pennie Preschool 1,"CSI-C, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,KL100335,NaN,Active,Henry Multi-Age,"EHS-A, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,KL100334,NaN,Active,Grampy Tom Multi-Age,"EHS-A, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,KL100337,NaN,Active,Henry Multi-Age,EHS-A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(244, 737)
Stage1: Renaming Column from 7 to 737 as JAN 01 to Dec 31


4,First Name,Last Name,Record ID,External Student ID,Student Status,Room,Tags,Jan 01 - IN,Jan 01 - OUT,Jan 02 - IN,...,Dec 27 - IN,Dec 27 - OUT,Dec 28 - IN,Dec 28 - OUT,Dec 29 - IN,Dec 29 - OUT,Dec 30 - IN,Dec 30 - OUT,Dec 31 - IN,Dec 31 - OUT
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IN,OUT,IN,...,IN,OUT,IN,OUT,IN,OUT,IN,OUT,IN,OUT
6,NaN,NaN,KL100349,NaN,Active,Pennie Preschool 1,"CSI-C, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,KL100335,NaN,Active,Henry Multi-Age,"EHS-A, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,KL100334,NaN,Active,Grampy Tom Multi-Age,"EHS-A, Vegetarian",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,KL100337,NaN,Active,Henry Multi-Age,EHS-A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(244, 737)
Stage2: Filter only Student Status = Active


...

(271, 120)
Stage5: Reshape with needed features 'Record ID', 'Room', 'Date', 'IN', 'OUT' 


,Record ID,Room,Date,IN,OUT
0,KL100393,Llamas Llamas,Jan 01 - IN,NaN,NaN
1,KL100393,Llamas Llamas,Jan 02 - IN,NaN,NaN
2,KL100393,Llamas Llamas,Jan 03 - IN,NaN,NaN
3,KL100393,Llamas Llamas,Jan 04 - IN,NaN,NaN
4,KL100393,Llamas Llamas,Jan 05 - IN,NaN,NaN


(15989, 5)
Stage6: Parse Timestamp from IN and OUT


,Record ID,Room,Date,IN,OUT
5,KL100393,Llamas Llamas,Jan 06 2025,9:20 AM,4:45 PM
6,KL100393,Llamas Llamas,Jan 07 2025,10:16 AM,4:29 PM
7,KL100393,Llamas Llamas,Jan 08 2025,9:52 AM,4:55 PM
8,KL100393,Llamas Llamas,Jan 09 2025,10:20 AM,4:30 PM
9,KL100393,Llamas Llamas,Jan 10 2025,9:05 AM,4:21 PM


,Record ID,Location,Room,AgeGroup,Date,Day of the Week,IN,OUT,Presence_0600,Presence_0630,...,Presence_1430,Presence_1500,Presence_1530,Presence_1600,Presence_1630,Presence_1700,Presence_1730,Presence_1800,Presence_1830,Presence_1900
0,KL100213,ECEC,Pre-K1,Pre-K,2022-01-03,Monday,09:04:00,17:07:00,0,0,...,1,1,1,1,1,1,0,0,0,0
1,KL100213,ECEC,Pre-K1,Pre-K,2022-01-04,Tuesday,08:49:00,17:15:00,0,0,...,1,1,1,1,1,1,0,0,0,0
2,KL100213,ECEC,Pre-K1,Pre-K,2022-01-05,Wednesday,09:04:00,17:30:00,0,0,...,1,1,1,1,1,1,0,0,0,0
3,KL100213,ECEC,Pre-K1,Pre-K,2022-01-06,Thursday,08:42:00,17:14:00,0,0,...,1,1,1,1,1,1,0,0,0,0
4,KL100213,ECEC,Pre-K1,Pre-K,2022-01-07,Friday,08:46:00,16:57:00,0,0,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131906,KL100413,Spellman,Wild Things,Preschool,2025-02-19,Wednesday,09:05:00,15:21:00,0,0,...,1,1,0,0,0,0,0,0,0,0
131907,KL100413,Spellman,Wild Things,Preschool,2025-02-24,Monday,08:53:00,15:48:00,0,0,...,1,1,1,0,0,0,0,0,0,0
131908,KL100413,Spellman,Wild Things,Preschool,2025-02-27,Thursday,08:34:00,15:06:00,0,0,...,1,1,0,0,0,0,0,0,0,0
131909,KL100413,Spellman,Wild Things,Preschool,2025-02-28,Friday,08:40:00,15:26:00,0,0,...,1,1,0,0,0,0,0,0,0,0


In [8]:
final_df

,Record ID,Location,Room,AgeGroup,Date,Day of the Week,IN,OUT,Presence_0600,Presence_0630,...,Presence_1430,Presence_1500,Presence_1530,Presence_1600,Presence_1630,Presence_1700,Presence_1730,Presence_1800,Presence_1830,Presence_1900
0,KL100213,ECEC,Pre-K1,Pre-K,2022-01-03,Monday,09:04:00,17:07:00,0,0,...,1,1,1,1,1,1,0,0,0,0
1,KL100213,ECEC,Pre-K1,Pre-K,2022-01-04,Tuesday,08:49:00,17:15:00,0,0,...,1,1,1,1,1,1,0,0,0,0
2,KL100213,ECEC,Pre-K1,Pre-K,2022-01-05,Wednesday,09:04:00,17:30:00,0,0,...,1,1,1,1,1,1,0,0,0,0
3,KL100213,ECEC,Pre-K1,Pre-K,2022-01-06,Thursday,08:42:00,17:14:00,0,0,...,1,1,1,1,1,1,0,0,0,0
4,KL100213,ECEC,Pre-K1,Pre-K,2022-01-07,Friday,08:46:00,16:57:00,0,0,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131906,KL100413,Spellman,Wild Things,Preschool,2025-02-19,Wednesday,09:05:00,15:21:00,0,0,...,1,1,0,0,0,0,0,0,0,0
131907,KL100413,Spellman,Wild Things,Preschool,2025-02-24,Monday,08:53:00,15:48:00,0,0,...,1,1,1,0,0,0,0,0,0,0
131908,KL100413,Spellman,Wild Things,Preschool,2025-02-27,Thursday,08:34:00,15:06:00,0,0,...,1,1,0,0,0,0,0,0,0,0
131909,KL100413,Spellman,Wild Things,Preschool,2025-02-28,Friday,08:40:00,15:26:00,0,0,...,1,1,0,0,0,0,0,0,0,0


In [14]:
# Decision Tree modeling at the 30 minute "Presence" intervals
presence_columns = [col for col in final_df.columns if 'Presence_' in col]

# grouping the data and summing it for the average attendence calculation
grouped_data = final_df.groupby(['Date', 'Day of the Week', 'Location', 'Room'])[presence_columns].sum().reset_index()

grouped_data

,Date,Day of the Week,Location,Room,Presence_0600,Presence_0630,Presence_0700,Presence_0730,Presence_0800,Presence_0830,...,Presence_1430,Presence_1500,Presence_1530,Presence_1600,Presence_1630,Presence_1700,Presence_1730,Presence_1800,Presence_1830,Presence_1900
0,2022-01-03,Monday,ECEC,Camp,0,0,0,0,1,2,...,2,2,2,2,2,0,0,0,0,0
1,2022-01-03,Monday,ECEC,Grampy Tom Multi-Age,0,0,0,0,1,1,...,1,1,1,1,1,1,0,0,0,0
2,2022-01-03,Monday,ECEC,Henry Toddlers,0,0,0,0,0,1,...,1,1,1,1,1,1,0,0,0,0
3,2022-01-03,Monday,ECEC,Pennie Preschool 1,0,0,0,2,4,4,...,7,7,7,7,5,3,1,0,0,0
4,2022-01-03,Monday,ECEC,Pennie Toddlers,0,0,0,0,1,1,...,2,2,2,2,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,2025-02-28,Friday,Spellman,Llamas Llamas,0,0,0,0,3,4,...,7,6,6,6,4,2,0,0,0,0
14846,2025-02-28,Friday,Spellman,Monkeys,0,0,0,3,5,8,...,11,9,9,9,7,6,3,1,0,0
14847,2025-02-28,Friday,Spellman,Pandas,0,0,0,2,3,7,...,9,9,9,9,9,1,0,0,0,0
14848,2025-02-28,Friday,Spellman,Rabbits,0,0,3,3,5,6,...,11,10,8,8,5,5,1,0,0,0


In [19]:
# Model Code

# creating the X and Y variables
X = grouped_data[['Location', 'Room', 'Day of the Week', 'Date']]
X = pd.get_dummies(X, columns=['Location', 'Room', 'Day of the Week'])
X['Date'] = X['Date'].apply(lambda date: date.toordinal())
y = grouped_data[presence_columns]

# splitting into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# setting up the model to predict values for each "presence" time interval column
models = {}
for col in presence_columns:
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train[col])
    models[col] = model

# predicting total attendance for each room, day and time interval
last_date = final_df['Date'].max()
next_week_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)

predictions = {}
for date in next_week_dates:
    day_of_week = date.day_name()
    for location in final_df['Location'].unique():
        for room in final_df['Room'].unique():
            new_data = pd.DataFrame({'Date': [date], 'Location': [location], 'Room': [room], 'Day of the Week': [day_of_week]})
            new_data = pd.get_dummies(new_data, columns=['Location', 'Room', 'Day of the Week'])
            new_data = new_data.reindex(columns=X_train.columns, fill_value=0)
            new_data['Date'] = new_data['Date'].apply(lambda date: date.toordinal())
            date_str = date.strftime('%Y-%m-%d')
            predictions.setdefault(date_str, {}).setdefault((location, room), {})
            for col in presence_columns:
                prediction = models[col].predict(new_data)[0]
                predictions[date_str][(location, room)][col] = prediction

# changing predictions to a dataframe
data = []

for date_str, location_room_data in predictions.items():
    for (location, room), presence_data in location_room_data.items():
        # adding back Age Group for Instructor to Student translations
        age_group = final_df.loc[final_df['Room'] == room, 'AgeGroup'].iloc[0]
        row_data = {'Date': date_str, 'Location': location, 'Room': room, 'AgeGroup': age_group}
        row_data.update(presence_data)
        data.append(row_data)

predictions_df = pd.DataFrame(data)

# translating attendance to staff numbers
ratios = {
    'Infant': 4,
    'Multi-Age': 4,
    'Toddler': 6,
    'Preschool': 10,
    'Pre-K': 12
}

for col in presence_columns:
    predictions_df[col.replace('Presence', 'Staff')] = predictions_df.apply(
        lambda row: math.ceil(row[col] / ratios[row['AgeGroup']]), axis=1
    )

# final model output
staff_columns = [col for col in predictions_df.columns if 'Staff_' in col]
column_order = ['Date', 'Location', 'Room', 'AgeGroup'] + staff_columns
Final_Forecast = predictions_df[column_order]

Final_Forecast

,Date,Location,Room,AgeGroup,Staff_0600,Staff_0630,Staff_0700,Staff_0730,Staff_0800,Staff_0830,...,Staff_1430,Staff_1500,Staff_1530,Staff_1600,Staff_1630,Staff_1700,Staff_1730,Staff_1800,Staff_1830,Staff_1900
0,2025-03-01,ECEC,Pre-K1,Pre-K,0,0,0,0,1,1,...,1,1,1,1,1,1,1,0,0,0
1,2025-03-01,ECEC,Henry Toddlers,Toddler,0,0,0,1,1,2,...,2,2,2,2,2,1,0,0,0,0
2,2025-03-01,ECEC,Pre-K2,Pre-K,0,0,0,1,1,1,...,1,1,1,1,1,1,1,0,0,0
3,2025-03-01,ECEC,Pennie Preschool 1,Preschool,0,0,0,1,1,1,...,1,1,1,1,1,1,1,0,0,0
4,2025-03-01,ECEC,Grampy Tom Preschool,Preschool,0,0,0,1,1,1,...,1,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2025-03-07,Spellman,Rainbow Fish,Preschool,0,0,0,0,0,1,...,1,1,1,1,1,1,0,0,0,0
304,2025-03-07,Spellman,House Pooh,Infant,0,0,1,1,1,2,...,2,2,2,2,1,1,1,0,0,0
305,2025-03-07,Spellman,Rabbits,Toddler,0,0,1,1,1,2,...,2,2,2,2,1,1,0,0,0,0
306,2025-03-07,Spellman,Hungry Caterpillars,Multi-Age,0,0,1,1,2,2,...,2,2,1,1,0,0,0,0,0,0


In [24]:
def visualize_forecast(df):
    df_melt = df.melt(id_vars=['Date', 'Location', 'Room', 'AgeGroup'],
                      var_name='Time_Interval',
                      value_name='Staff_Needed')

    fig = px.bar(df_melt,
                 x='Time_Interval',
                 y='Staff_Needed',
                 color='AgeGroup',
                 facet_col='Room',
                 facet_row='Location',
                 animation_frame='Date',
                 labels={'Staff_Needed': 'Staff Needed', 'Time_Interval': 'Time Interval'},
                 title='Staffing Forecast per Location, Room and Age Group')

    fig.update_layout(height=1000, width=1200)
    fig.show()

visualize_forecast(Final_Forecast)
